In [5]:
SAMPLES_DIR="../samples/"
ETC_DIR=SAMPLES_DIR+"etc/"
AUDIO_DIR=SAMPLES_DIR+"audio/"
OUT_DIR=SAMPLES_DIR+"out/"
VIDEO_NAME="Untitled.mp4"
AUDIO_NAME="Untitled.wav"

In [6]:
from pydub import AudioSegment

video = AudioSegment.from_file(SAMPLES_DIR+VIDEO_NAME, format="mp4")

In [7]:
audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)

In [8]:
audio.export(AUDIO_DIR+AUDIO_NAME, format="wav")

<_io.BufferedRandom name='../samples/audio/Untitled.wav'>

In [9]:
import whisper
model = whisper.load_model("base")

In [10]:
output = model.transcribe(AUDIO_DIR+AUDIO_NAME)
print(output)

/opt/miniconda3/envs/sportlight/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': " I just about walking to the middle, hoping to make a point. They'll be wanting to make a point, that's for sure, but isn't it ironic? The fact that it's Gilchrist Hayden, they're both mid-30s, and we were talking about the big three that aren't there for India. Could Tendor could be a master at this game as well. But all I know is this is an exciting day for Indian cricket the first time they've got a one day or a 2020 international. And as soon as Australia had clinched the future cup in Nagpur, all Indians have been waiting for this day. And so here it is, I suppose everyone has just a sit back and enjoy it. Right, wait no longer as we move across and join our commentators to watch Australia that first. Yeah, that's right, always well fed a sack. Yeah, I know what you mean. Also, I hadn't 30 of birth, I hear one approve himself. But there's Adam Gilchrist being around a long time now. He knows the format of this game. Interesting field. There they are. They've got two slip

In [11]:
positive = list()

add = ["Straight", "biggie", "Cover", "OnDrive", "Square", "Forward", "stadium", "Defence", "Sweep", "Reverse",
           "FrontFoot ", "LegGlance ", "BackFoot", "SquareCut", "Pull ", "Shot", "Hook", "Uppercut", "Cut", "Helicopter ", "SwitchHit",
           "Dilscoop", "class", "bounce", "Upper", "Uppish", "Scoop ", "Inside", "Out", "Shots", "Bouncer", "Outswinger", "Inswinger",
           "ReverseSwing", "played", "LegCutter", "OffCutter", "Yorker", "Slower", "Spin", "LegBreak ", "OffBreak", "Googly ",
           "Doosra", "Topspin ", "CarromBall", "Slider", "ArmBall", "Infield", "InnerRing", "Outfield", "Catching", "Wicketkeeper",
           "Slip", "Gully", "LegSlip", "LegGully", "Sillypoint", "Sillymidoff", "Shortleg", "Sillymidon", "InnerRing", "Point", "BackwardPoint",
           "MidOff", "Cover", "MidOn", "SquareLeg", "Backward ", "SquareLeg", "MidWicket", "FineLeg", "Outfield", "ThirdMan",
           "DeepPoint", "BackwardPoint", "ExtraCover", "LongOff", "FineLeg", "LongLeg", "LongOn", "Deep", "Cover", "played", "account"
           "cricket", "hard", "sides", "man", "finishes", "one", "crucial", "Captain", "shot", "six", "four", "boundary", "line", "drive",
           "celebrate", "placement", "beauty", "fifty", "century", "perfect", "magnifcient", "world", "cup", "batting", "fielding", "bowling",
           "catch", "caught", "out", "stumped", "one", "bowled", "night", "final", "room", "taken", "edged", "wicket", "review", "DRS", "cuts", "out", "short"]

In [12]:
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arjunsivaraman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/arjunsivaraman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
from nltk.corpus import wordnet

for i in add:
        for synset in wordnet.synsets(i):
            for lemma in synset.lemmas():
                positive.append(lemma.name())

In [14]:
strings = ' '.join(positive)

In [15]:
data = output['segments']

In [16]:
text = []
start_time = []
end_time = []

for i in data:
    text.append(i['text'])
    start_time.append(i['start'])
    end_time.append(i['end'])

In [17]:
import pandas as pd

def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return (df)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

score = []
for i in text:
    d = [i, strings]
    Tfidf_vect = TfidfVectorizer()
    vector_matrix = Tfidf_vect.fit_transform(d)
    tokens = Tfidf_vect.get_feature_names_out()
    create_dataframe(vector_matrix.toarray(), tokens)
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    result = create_dataframe(
        cosine_similarity_matrix, ['Phrase', 'Strings'])
    score.append(result['Phrase'].values[1])

In [24]:
stack_first = []
stack_last = []

for i in range(len(score)):
    if (score[i] >= 0.01000000000000):
        stack_first.append(start_time[i])
        stack_last.append(end_time[i])

In [25]:
# Sort both lists based on one list
# stack_first, stack_last = zip(*sorted(zip(stack_first, stack_last)))
result = sorted(zip(stack_first, stack_last))

In [26]:
result

[(0.0, 4.0),
 (4.0, 7.92),
 (19.76, 22.32),
 (51.2, 54.2),
 (71.2, 73.2),
 (76.2, 78.2),
 (88.2, 91.2),
 (98.2, 103.2),
 (104.2, 106.2),
 (131.2, 133.2),
 (152.2, 153.2),
 (170.2, 171.2),
 (173.2, 175.2),
 (187.2, 190.2),
 (192.2, 196.2),
 (201.2, 204.2),
 (216.2, 219.2),
 (227.2, 229.2),
 (243.2, 244.2),
 (258.2, 260.2),
 (270.2, 273.2),
 (278.2, 282.2),
 (290.2, 293.2),
 (301.2, 302.2),
 (314.2, 316.2),
 (334.2, 336.2),
 (337.2, 338.2),
 (341.2, 342.2),
 (350.2, 353.2),
 (366.2, 367.2),
 (393.2, 396.2),
 (454.2, 456.2),
 (476.2, 478.2),
 (492.2, 493.2),
 (494.2, 496.2),
 (501.2, 502.2),
 (507.2, 509.2),
 (509.2, 512.2),
 (516.2, 520.2),
 (545.2, 546.2),
 (546.2, 550.2),
 (550.2, 553.2),
 (570.2, 573.2),
 (609.2, 613.2),
 (625.2, 627.2),
 (628.2, 632.2),
 (646.2, 650.2),
 (668.2, 672.2),
 (690.2, 691.2),
 (697.2, 701.2),
 (701.2, 704.2),
 (707.2, 708.2),
 (708.2, 709.2),
 (713.2, 715.2),
 (721.2, 724.2),
 (750.2, 752.2),
 (752.2, 754.2),
 (767.2, 771.2),
 (785.2, 787.2),
 (824.2, 826.

In [27]:
res = [[result[0][0] - 4 if result[0][0] - 4 > 0 else 0, result[0][1] + 3]]
for i in range(1, len(result)):
    if res[-1][1] + 3 >= result[i][0] - 4:
        if res[-1][1] + 3 < result[i][1] + 3:
            res[-1][1] = result[i][1] + 3
    else:
        res.append([result[i][0] - 4 if result[i][0] - 4 > 0 else 0, result[i][1] + 3])

In [28]:
res

[[0, 10.92],
 [15.760000000000002, 25.32],
 [47.2, 57.2],
 [67.2, 106.2],
 [127.19999999999999, 136.2],
 [148.2, 156.2],
 [166.2, 178.2],
 [183.2, 207.2],
 [212.2, 232.2],
 [239.2, 247.2],
 [254.2, 305.2],
 [310.2, 319.2],
 [330.2, 356.2],
 [362.2, 370.2],
 [389.2, 399.2],
 [450.2, 459.2],
 [472.2, 481.2],
 [488.2, 523.2],
 [541.2, 553.2],
 [566.2, 576.2],
 [605.2, 616.2],
 [621.2, 635.2],
 [642.2, 653.2],
 [664.2, 675.2],
 [686.2, 727.2],
 [746.2, 755.2],
 [763.2, 774.2],
 [781.2, 790.2],
 [820.2, 836.2],
 [840.2, 860.2],
 [880.2, 889.2],
 [897.2, 906.2],
 [929.2, 938.2],
 [962.2, 979.2],
 [987.2, 996.2],
 [1000.2, 1029.2],
 [1036.2, 1045.2],
 [1056.2, 1069.2],
 [1073.2, 1088.2],
 [1097.2, 1105.2],
 [1133.2, 1150.2],
 [1170.2, 1196.2],
 [1205.2, 1238.2],
 [1293.2, 1301.2],
 [1310.2, 1369.2],
 [1413.2, 1421.2],
 [1426.2, 1458.2],
 [1497.2, 1533.2]]

In [29]:
#Create temporary folder for storing subclips generated. This folder will be deleted later after highlights are generated. 
import os, shutil
sub_folder=os.path.join(ETC_DIR,"Subclips")
if os.path.exists(sub_folder):
	shutil.rmtree(sub_folder)
	path=os.mkdir(sub_folder)
else:
	path=os.mkdir(sub_folder)
#print(sub_folder,type(sub_folder))

In [31]:
#Extract moments from videos to be added in highlight
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip, concatenate_videoclips
for i in range(len(res)):
	filename="highlight" + str(i+1) + ".mp4"
	ffmpeg_extract_subclip(SAMPLES_DIR+VIDEO_NAME,res[i][0],res[i][1],targetname=sub_folder+"/"+filename) #Enter your sports video clip name here.

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [33]:
files=os.listdir(sub_folder)
files=[sub_folder+"/highlight" + str(i+1) + ".mp4" for i in range(len(res))]
#print(files)
final_clip=concatenate_videoclips([VideoFileClip(i) for i in files])
final_clip.write_videofile(OUT_DIR+VIDEO_NAME) #Enter the desired output highlights filename.
shutil.rmtree(sub_folder) #Delete the temporary file.

Moviepy - Building video ../samples/out/Untitled.mp4.
MoviePy - Writing audio in UntitledTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../samples/out/Untitled.mp4



Moviepy - Done !
Moviepy - video ready ../samples/out/Untitled.mp4
